In [2]:
!pip install pandas
!pip install numpy
!pip install openpyxl


                   

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


In [1]:
import pandas as pd
import numpy as np
### Users

user_gameplay_path = r"C:\Users\Praveen\user_gameplay_202211281119.csv"
deposit_gameplay_path = r"C:\Users\Praveen\Downloads\deposit.csv"
withdraw_gameplay_path = r"C:\Users\Praveen\Downloads\withdraw.csv.csv"
df_users = pd.read_csv(user_gameplay_path)
df_users = df_users.rename(columns={"USER ID":"User Id"})
df_users['Datetime'] = pd.to_datetime(df_users['Datetime'], format='%d-%m-%Y %H:%M')

## Deposit
df_deposit = pd.read_csv(deposit_gameplay_path)
df_deposit['Datetime'] = pd.to_datetime(df_deposit['Datetime'], format='%d-%m-%Y %H:%M')

## withdraw

df_withdraw = pd.read_csv(withdraw_gameplay_path)
df_withdraw['Datetime'] = pd.to_datetime(df_withdraw['Datetime'], format='%d-%m-%Y %H:%M')
df_users


,User Id,Games Played,Datetime
0,851,1,2022-10-01 00:00:00
1,717,1,2022-10-01 00:00:00
2,456,1,2022-10-01 00:00:00
3,424,1,2022-10-01 00:00:00
4,845,1,2022-10-01 00:00:00
...,...,...,...
354995,154,1,2022-10-31 23:26:00
354996,449,1,2022-10-31 23:26:00
354997,677,1,2022-10-31 23:26:00
354998,765,1,2022-10-31 23:26:00


In [54]:


## Date format yy-mm-dd example 2022-10-02 
def generate_loyality_points(date : str , slot : str ,month_start='',month_end=''):
    if slot.lower()=='s1':
        date_start = date + " 00:00"
        date_end = date + " 12:00"
    elif slot.lower()=='s2':
        date_start = date + " 12:00"
        date_end = date + " 23:59"
    else:
        date_start = month_start
        date_end = month_end
        
    
    df_users_temp1 = df_users[(df_users['Datetime'] > date_start) & (df_users['Datetime'] < date_end)]
    df_deposit_temp1 = df_deposit[(df_deposit['Datetime'] > date_start) & (df_deposit['Datetime'] < date_end)]
    df_withdraw_temp1 = df_withdraw[(df_withdraw['Datetime'] > date_start) & (df_withdraw['Datetime'] < date_end)]



    df_deposit_temp1 = df_deposit_temp1.groupby(['User Id'],as_index=False)['Amount'].sum()

    df_all_deposit = df_deposit_temp1.rename(columns={'Amount':'deposit_amount'})

    df_withdraw_temp1 = df_withdraw_temp1.groupby('User Id',as_index=False)['Amount'].sum()
    df_all_withdraw = df_withdraw_temp1.rename(columns={'Amount':'withdraw_amount'})
    df_all_users = df_users_temp1.groupby('User Id',as_index=False)['Games Played'].sum()

    df_all_merge = pd.merge(df_all_users,df_all_withdraw,on='User Id',how='left')
    df_all_merge = pd.merge(df_all_merge,df_all_deposit,on='User Id',how='left')
    df_all_merge = df_all_merge.fillna(0)

    df_all_merge['points'] = df_all_merge.apply(lambda x: (0.01*x.deposit_amount+0.005*x.withdraw_amount+0.2*x['Games Played']
                                                         + 0.001*(max(x.deposit_amount-x.withdraw_amount,0))),axis=1)
    df_all_merge=df_all_merge.sort_values(by=['points'],ascending=False)
    return df_all_merge

In [55]:
df_final_oct_02_s1 = generate_loyality_points(date='2022-10-02',slot='s1')
df_final_oct_16_s2 = generate_loyality_points(date='2022-10-16',slot='s2')
df_final_oct_18_s1 = generate_loyality_points(date='2022-10-18',slot='s1')
df_final_oct_26_s2 = generate_loyality_points(date='2022-10-26',slot='s2')
df_final_oct = generate_loyality_points(date='' , slot='all' ,month_start='2022-10-01 00:00',month_end='2022-10-31 23:59')
df_final_oct_02_s1.to_excel('df_final_oct_02_s1.xlsx',index=False)
df_final_oct_16_s2.to_excel('df_final_oct_16_s2.xlsx',index=False)
df_final_oct_18_s1.to_excel('df_final_oct_18_s1.xlsx',index=False)
df_final_oct_26_s2.to_excel('df_final_oct_26_s2.xlsx',index=False)

## Caluclating rank for users who has highest points 
df_final_oct=df_final_oct.sort_values(by=['points','Games Played'],ascending=[False,False])

df_final_oct['rank'] = np.arange(1,df_final_oct.shape[0]+1)
print(df_final_oct)
df_final_oct.to_excel('df_final_oct.xlsx',index=False)

## Avg Depoit amount 
df_avg_deposit = df_deposit['Amount'].mean()

print('Avg Depoit amount' , df_avg_deposit)

## Avg deposit amount per month per user
import datetime
df_deposit['Month'] = df_deposit['Datetime'].apply(lambda x : x.strftime("%B"))
df_avg_depoist_user = df_deposit.groupby(['User Id','Month'],as_index=False)['Amount'].mean()
print(df_avg_depoist_user)

## Avg Number of games played per user

df_avg_user = df_users.groupby(['User Id'],as_index=False)['Games Played'].mean()
print(df_avg_user)


     User Id  Games Played  withdraw_amount  deposit_amount     points  rank
634      634            24       15737705.0        515000.0  83843.325     1
672      672            10         233750.0       2158700.0  24682.700     2
99        99            10        2403141.0       1164800.0  23665.705     3
212      212             1         589850.0       1924981.0  23534.391     4
566      566           183         185071.0       1819175.0  20787.809     5
..       ...           ...              ...             ...        ...   ...
858      858             2              0.0             0.0      0.400   996
643      643             2              0.0             0.0      0.400   997
388      388             2              0.0             0.0      0.400   998
507      507             2              0.0             0.0      0.400   999
38        38             1              0.0             0.0      0.200  1000

[1000 rows x 6 columns]
Avg Depoit amount 5492.185399701801
     User Id   

In [56]:
## Alloting loyality amount to top fifty palyers
df_final_oct = df_final_oct.head(50)

sum_of_points = df_final_oct['points'].sum()
sum_of_games_played = df_final_oct['Games Played'].sum()
each_point_value = 50000/sum_of_points
each_gp_value = 50000/sum_of_games_played

df_final_oct['prize_money'] = df_final_oct.apply(lambda x : (x['points']*each_point_value+x['Games Played']*each_gp_value)/2 ,axis=1)


df_final_oct.to_excel('df_final_oct_prize_money.xlsx',index=False)
df_final_oct

,User Id,Games Played,withdraw_amount,deposit_amount,points,rank,prize_money
634,634,24,15737705.0,515000.0,83843.325,1,3220.364479
672,672,10,233750.0,2158700.0,24682.700,2,950.426070
99,99,10,2403141.0,1164800.0,23665.705,3,911.600138
212,212,1,589850.0,1924981.0,23534.391,4,899.286258
566,566,183,185071.0,1819175.0,20787.809,5,942.065889
740,740,2,365288.0,1738490.0,20584.942,6,787.496002
714,714,6,0.0,1676300.0,18440.500,7,708.872152
421,421,1555,1269809.0,878600.0,15446.045,8,1851.082573
30,30,13,152145.0,1329000.0,15230.180,9,591.989723
222,222,10,99358.0,1285000.0,14534.432,10,562.994510
